In [1]:
import tensorflow as tf
from model import DCGAN
from ops import *
from utils import *
from tensorflow.examples.tutorials.mnist import input_data

/home/lemac/miniconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
# Inits
batch_size = 64
y_dim = 10
c_dim = 1
output_height = 28
output_width = 28
df_dim=64
dfc_dim=1024
z_dim = 100
# z = tf.placeholder(tf.float32, [None, z_dim], name='z')
# y = tf.placeholder(tf.float32, [batch_size, y_dim], name='y')                         # (64, 10)
# d_bn1 = batch_norm(name='d_bn1')
# d_bn2 = batch_norm(name='d_bn2')
sess = tf.Session()

In [ ]:
z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')
#first deconv block
g_w1 = tf.get_variable('g_w1', [z_dim,102400 ], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.02))
g_b1 = tf.get_variable('g_b1', [102400], initializer=tf.truncated_normal_initializer(stddev=0.02))
g1 = tf.matmul(z, g_w1) + g_b1
g1 = tf.reshape(g1, [-1, 20, 20, 256])
g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
g1 = tf.nn.relu(g1)
print(g1.get_shape())
#CapsNet Implementation

In [ ]:
# Primary Capsules layer, return [batch_size, 1152, 8, 1]
primaryCaps = CapsConv2(num_units=8, with_routing=False)

In [ ]:
caps1Gen = primaryCaps(sess, g1, batch_size, num_outputs=256, kernel_size=9, stride=2)

In [ ]:
# DigitCaps layer, [batch_size, 10, 16, 1]
digitCaps = CapsConv2(num_units=16, with_routing=True)

In [ ]:
caps2Gen = digitCaps(sess, caps1Gen, batch_size, num_outputs=10)

In [ ]:
# Decoder structure in Fig. 2
# 1. Do masking, how:
    # a). calc ||v_c||, then do softmax(||v_c||)
    # [batch_size, 10, 16, 1] => [batch_size, 10, 1, 1]
v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen),
                        axis=2, keep_dims=True) + 1e-9)
softmax_v = tf.nn.softmax(v_length, dim=1)
assert softmax_v.get_shape() == [64, 10, 1, 1]

# b). pick out the index of max softmax val of the 10 caps
# [batch_size, 10, 1, 1] => [batch_size] (index)
argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis=1))
assert argmax_idx.get_shape() == [64, 1, 1]
argmax_idx = tf.reshape(argmax_idx, shape=(64, ))

# Method 1.
if not True:
    # c). indexing
    # It's not easy to understand the indexing process with argmax_idx
    # as we are 3-dim animal
    masked_v = []
    for batch_size in range(64):
        v = caps2Gen[batch_size][argmax_idx[batch_size], :]
        masked_v.append(tf.reshape(v, shape=(1, 1, 16, 1)))

    masked_v = tf.concat(masked_v, axis=0)
    assert masked_v.get_shape() == [64, 1, 16, 1]
# Method 2. masking with true label, default mode
else:
    # masked_v = tf.matmul(tf.squeeze(caps2), tf.reshape(Y, (-1, 10, 1)), transpose_a=True)
    masked_v = tf.multiply(tf.squeeze(caps2Gen), tf.reshape(np.random.uniform(0.0,9.0,size=10).astype(np.float32), (-1, 10, 1)))
    v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen), axis=2, keep_dims=True) + 1e-9)

In [ ]:
# 2. Reconstructe the MNIST images with 3 FC layers
# [batch_size, 1, 16, 1] => [batch_size, 16] => [batch_size, 512]
vector_j = tf.reshape(caps2Gen, shape=(64, -1))
fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
assert fc1.get_shape() == [64, 512]
fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
assert fc2.get_shape() == [64, 1024]

output = tf.contrib.layers.fully_connected(fc2, num_outputs=784, activation_fn=tf.sigmoid)


In [ ]:
output

In [ ]:
class CapsConv(object):
    ''' Capsule layer.
    Args:
        input: A 4-D tensor.
        num_units: integer, the length of the output vector of a capsule.
        with_routing: boolean, this capsule is routing with the
        lower-level layer capsule.
        num_outputs: the number of capsule in this layer.
    Returns:
        A 4-D tensor.
    '''
    def __init__(self, num_units, with_routing=True):
        self.num_units = num_units
        self.with_routing = with_routing
    def __call__(self, input, num_outputs, batch_size, kernel_size=None, stride=None):
        self.num_outputs = num_outputs
        self.kernel_size = kernel_size
        self.stride = stride
        self.batch_size = batch_size
        if not self.with_routing:
            #assert input.get_shape() == [None, 28,28,256]

            capsules = tf.contrib.layers.conv2d(input, self.num_outputs,
                            self.kernel_size, self.stride, padding="VALID",
                            activation_fn=tf.nn.relu)
            capsules = tf.reshape(capsules, (self.batch_size, -1, self.num_units, 1))

            # [batch_size, 1152, 8, 1]
            capsules = squash(capsules)
            print(capsules.get_shape())
            assert capsules.get_shape() == [self.batch_size, 1152, 8, 1]
            return(capsules)
        else:
            # the DigitCap layer
            # reshape the input into shape [128,1152,8,1]
            input = tf.reshape(input, shape=(self.batch_size, 1152, 8,1))

            #b_IJ : [1, num_caps_1, num_caps_1_plus_1, 1]
            b_IJ = tf.zeros(shape=[1,1152,10,1], dtype=np.float32)
            capsules = []
            for j in range(self.num_outputs):
                with tf.variable_scope('caps_' + str(j)):
                    caps_j, b_IJ = capsule(input, b_IJ, j)
                    capsules.append(caps_j)

            #return a tensor with shape [batch_size,10,16,1]
            capsules = tf.concat(capsules, axis=1)
            assert capsules.get_shape() == [self.batch_size,10,16,1]

        return(capsules)

class CapsConv2(object):
    ''' Capsule layer.
    Args:
        input: A 4-D tensor.
        num_units: integer, the length of the output vector of a capsule.
        with_routing: boolean, this capsule is routing with the
        lower-level layer capsule.
        num_outputs: the number of capsule in this layer.
    Returns:
        A 4-D tensor.
    '''
    def __init__(self, num_units, with_routing=True):
        self.num_units = num_units
        self.with_routing = with_routing
    def __call__(self, input, batch_size, num_outputs, kernel_size=None, stride=None):
        self.num_outputs = num_outputs
        self.kernel_size = kernel_size
        self.stride = stride
        self.batch_size = batch_size
        if not self.with_routing:
            #assert input.get_shape() == [None, 28,28,256]

            capsules = tf.contrib.layers.conv2d(input, self.num_outputs,
                        self.kernel_size, self.stride, padding="VALID",
                        activation_fn=tf.nn.relu)
            capsules = tf.reshape(capsules, (self.batch_size, -1, self.num_units, 1))

            # [batch_size, 1152, 8, 1]
            capsules = squash(capsules)
            print(capsules.get_shape())
            assert capsules.get_shape() == [self.batch_size, 1152, 8, 1]
            return(capsules)

        else:
            # the DigitCap layer
            # reshape the input into shape [128,1152,8,1]
            input = tf.reshape(input, shape=(self.batch_size, 1152, 8,1))

            #b_IJ : [1, num_caps_1, num_caps_1_plus_1, 1]
            b_IJ = tf.zeros(shape=[1,1152,10,1], dtype=np.float32)
            capsules = []
            for j in range(self.num_outputs):
                with tf.variable_scope('caps_' + str(j)):
                    caps_j, b_IJ = capsule(input, b_IJ, j)
                    capsules.append(caps_j)

                    #return a tensor with shape [batch_size,10,16,1]
                    capsules = tf.concat(capsules, axis=1)
                    assert capsules.get_shape() == [self.batch_size,10,16,1]

        return(capsules)

def capsule(input, b_IJ, idx_j):
    ''' The routing algorithm for one capsule in the layer l+1.
    Args:
        input: A Tensor with [batch_size, num_caps_l=1152, length(u_i)=8, 1]
        shape, num_caps_l meaning the number of capsule in the layer l.
    Returns:
        A Tensor of shape [batch_size, 1, length(v_j)=16, 1] representing the
        vector output `v_j` of capsule j in the layer l+1
    Notes:
        u_i represents the vector output of capsule i in the layer l, and
        v_j the vector output of capsule j in the layer l+1.
    '''
    with tf.variable_scope('routing'):

        w_initializer = np.random.normal(size=[1, 1152, 8, 16], scale=0.01)

        W_Ij = tf.Variable(w_initializer, dtype=tf.float32)
        sess.run(tf.global_variables_initializer())
        # repeat W_Ij with batch_size times to shape [batch_size, 1152, 8, 16]
        W_Ij = tf.tile(W_Ij, [64, 1, 1, 1])

        # calc u_hat
        # [8, 16].T x [8, 1] => [16, 1] => [batch_size, 1152, 16, 1]
        u_hat = tf.matmul(W_Ij, input, transpose_a=True)
        assert u_hat.get_shape() == [64, 1152, 16, 1]

        shape = b_IJ.get_shape().as_list()
        size_splits = [idx_j, 1, shape[2] - idx_j - 1]
        for r_iter in range(3):
            # line 4:
            # [1, 1152, 10, 1]
            c_IJ = tf.nn.softmax(b_IJ, dim=2)
            assert c_IJ.get_shape() == [1, 1152, 10, 1]

            # line 5:
            # weighting u_hat with c_I in the third dim,
            # then sum in the second dim, resulting in [batch_size, 1, 16, 1]
            b_Il, b_Ij, b_Ir = tf.split(b_IJ, size_splits, axis=2)
            c_Il, c_Ij, b_Ir = tf.split(c_IJ, size_splits, axis=2)
            assert c_Ij.get_shape() == [1, 1152, 1, 1]

            s_j = tf.multiply(c_Ij, u_hat)
            s_j = tf.reduce_sum(tf.multiply(c_Ij, u_hat),
                                axis=1, keep_dims=True)
            assert s_j.get_shape() == [64, 1, 16, 1]

            # line 6:
            # squash using Eq.1, resulting in [batch_size, 1, 16, 1]
            v_j = squash(s_j)
            assert s_j.get_shape() == [64, 1, 16, 1]

            # line 7:
            # tile v_j from [batch_size ,1, 16, 1] to [batch_size, 1152, 16, 1]
            # [16, 1].T x [16, 1] => [1, 1], then reduce mean in the
            # batch_size dim, resulting in [1, 1152, 1, 1]
            v_j_tiled = tf.tile(v_j, [1, 1152, 1, 1])
            u_produce_v = tf.matmul(u_hat, v_j_tiled, transpose_a=True)
            assert u_produce_v.get_shape() == [64, 1152, 1, 1]
            b_Ij += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
            b_IJ = tf.concat([b_Il, b_Ij, b_Ir], axis=2)

        return(v_j, b_IJ)

def squash(vector):
    '''Squashing function.
    Args:
        vector: A 4-D tensor with shape [batch_size, num_caps, vec_len, 1],
    Returns:
        A 4-D tensor with the same shape as vector but
        squashed in 3rd and 4th dimensions.
    '''
    vec_abs = tf.sqrt(tf.reduce_sum(tf.square(vector)))  # a scalar
    scalar_factor = tf.square(vec_abs) / (1 + tf.square(vec_abs))
    vec_squashed = scalar_factor * tf.divide(vector, vec_abs)  # element-wise
    return(vec_squashed)


In [ ]:
# KEEP DEBUGGING

### Testing CapsConv2 standalone --done

In [ ]:
# Inits
batch_size = 64
y_dim = 10
c_dim = 1
output_height = 28
output_width = 28
df_dim=64
dfc_dim=1024
z_dim = 100

z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')
#first deconv block
g_w1 = tf.get_variable('g_w1', [z_dim,102400 ], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.02))
g_b1 = tf.get_variable('g_b1', [102400], initializer=tf.truncated_normal_initializer(stddev=0.02))
g1 = tf.matmul(z, g_w1) + g_b1
g1 = tf.reshape(g1, [-1, 20, 20, 256])
g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
g1 = tf.nn.relu(g1)
print(g1.get_shape())
#CapsNet Implementation

In [ ]:
num_units = 8
x = g1 
num_outputs = 256
kernel_size = 9
stride = 2

In [ ]:
capsules = tf.contrib.layers.conv2d(g1, num_outputs,
                                    kernel_size, stride, padding="VALID",
                                    activation_fn=tf.nn.relu)

In [ ]:
capsules = tf.reshape(capsules, (64, -1, num_units, 1))

In [ ]:
# [batch_size, 1152, 8, 1]
capsules = squash(capsules)
print(capsules.get_shape())
assert capsules.get_shape() == [64, 1152, 8, 1]
output = capsules

### Comparing generators

In [3]:
batch_size = 64
z_dim = 100
sess = tf.Session()

In [ ]:
z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')
#first deconv block
g_w1 = tf.get_variable('g_w1', [z_dim,102400 ], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.02))
g_b1 = tf.get_variable('g_b1', [102400], initializer=tf.truncated_normal_initializer(stddev=0.02))
g1 = tf.matmul(z, g_w1) + g_b1
g1 = tf.reshape(g1, [-1, 20, 20, 256])
g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
g1 = tf.nn.relu(g1)
print(g1.get_shape())
#CapsNet Implementation

# Primary Capsules layer, return [batch_size, 1152, 8, 1]
primaryCaps = CapsConv2(num_units=8, with_routing=False)
caps1Gen = primaryCaps(sess, g1, num_outputs=256, batch_size=batch_size, kernel_size=9, stride=2)
#assert caps1.get_shape() == [128, 1152, 8, 1]

# DigitCaps layer, [batch_size, 10, 16, 1]
digitCaps = CapsConv2(num_units=16, with_routing=True)
caps2Gen = digitCaps(sess, caps1Gen, num_outputs=10, batch_size=batch_size)


# Decoder structure in Fig. 2
# 1. Do masking, how:
# a). calc ||v_c||, then do softmax(||v_c||)
# [batch_size, 10, 16, 1] => [batch_size, 10, 1, 1]
v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen),
                                      axis=2, keep_dims=True) + 1e-9)
softmax_v = tf.nn.softmax(v_length, dim=1)
assert softmax_v.get_shape() == [64, 10, 1, 1]

# b). pick out the index of max softmax val of the 10 caps
# [batch_size, 10, 1, 1] => [batch_size] (index)
argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis=1))
assert argmax_idx.get_shape() == [64, 1, 1]
argmax_idx = tf.reshape(argmax_idx, shape=(64, ))

# Method 1.
if not True:
    # c). indexing
    # It's not easy to understand the indexing process with argmax_idx
    # as we are 3-dim animal
    masked_v = []
    for batch_size in range(64):
        v = caps2Gen[batch_size][argmax_idx[batch_size], :]
        masked_v.append(tf.reshape(v, shape=(1, 1, 16, 1)))

    masked_v = tf.concat(masked_v, axis=0)
    assert masked_v.get_shape() == [64, 1, 16, 1]
# Method 2. masking with true label, default mode
else:
    # masked_v = tf.matmul(tf.squeeze(caps2), tf.reshape(Y, (-1, 10, 1)), transpose_a=True)
    masked_v = tf.multiply(tf.squeeze(caps2Gen), tf.reshape(np.random.uniform(0.0,9.0,size=10).astype(np.float32), (-1, 10, 1)))
    v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2Gen), axis=2, keep_dims=True) + 1e-9)


# 2. Reconstructe the MNIST images with 3 FC layers
# [batch_size, 1, 16, 1] => [batch_size, 16] => [batch_size, 512]
vector_j = tf.reshape(caps2Gen, shape=(64, -1))
fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
assert fc1.get_shape() == [64, 512]
fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
assert fc2.get_shape() == [64, 1024]
decoded = tf.contrib.layers.fully_connected(fc2, num_outputs=784, activation_fn=tf.sigmoid)

In [5]:
decoded

<tf.Tensor 'fully_connected_2/Sigmoid:0' shape=(64, 784) dtype=float32>

In [8]:
tf.reshape(decoded, shape=(64, 28, 28, 1))

<tf.Tensor 'Reshape_10:0' shape=(64, 28, 28, 1) dtype=float32>